In [3]:
import tweepy, os, sys

sys.path.append(os.path.abspath("../../../src/"))
from helper.utils import *
from helper.convmanager import *

credentials = Credentials('../../../res/credentials.yml')

### Twitter API Instance
To create a Twitter API instance, you need four pieces of information:
- CONSUMER_KEY
- CONSUMER_SECRET
- ACCESS_TOKEN
- ACCESS_TOKEN_SECRET

In [ ]:
auth = tweepy.OAuthHandler(
    credentials.twitter['CONSUMER_KEY'],
    credentials.twitter['CONSUMER_SECRET'])
auth.set_access_token(
    credentials.twitter['ACCESS_TOKEN'],
    credentials.twitter['ACCESS_TOKEN_SECRET'])

API = tweepy.API(auth)

### Search API
Returns a collection of relevant Tweets matching a specified query. Relevant params:
- **q**: query, relevant argument of the retrieved tweets
- **lang**: language of the retrieved tweets
- **tweet_mode**: if true, text does not get truncated



Then with ```Cursor``` method of tweepy, you can retrieve up to **1000** tweets related to that query.

In [4]:
search_options = {
    'q':          'christmas',  # query
    'lang':       'en',         # language
    'tweet_mode': 'extended'    # get full tweet text
}

statuses = tweepy.Cursor(api.search, **search_options).items(1000) 

### Build the conversation
The goal is to rebuild a conversation, because the 1000 tweets retrieved may be unrelated. The steps are:
1. Filter tweets with no parent tweet
2. Starting from each remaining tweet, make another API request to retrieve the parent tweet (the tweet to which the current one is responding), if not already present, and then proceed recursively
3. You end up with potentially up to 1000 conversations, so you can try to merge some of them if they have elements in common.

#### - Step 1: Filter tweets with no parent tweet

In [ ]:
tweets = [[convert(status)] for status in statuses if is_response(status)]

#### - Step 2: Extend the conversations

In [ ]:
convs  = [extend(api, tweet) for tweet in tweets]

#### - Step 3: Merge the conversations

In [ ]:
merged_convs = merge_conversations(convs, has_common=has_common_tweet)